# EEG SNR Batch
This notebook takes multiple `.npy` and `.json` files and computes the modified SNR for all the epochs in the `npy` file.

## Import all necessary packages

In [1]:
# Standard libraries
import json
import numpy as np
import xlsxwriter
import pandas as pd
import scipy.signal as signal
import matplotlib.pyplot as plt

# Custom libraries
from Functions import processing

## Load data

In [136]:
# List of file names to be processed
files = [  
    "sub-P008_ses-S002_task-T1_run-001_eeg"   
]

#  "sub-P003_ses-S001_task-T1_run-001_eeg",
#    "sub-P003_ses-S002_task-T1_run-001_eeg",
#    "sub-P004_ses-S001_task-T1_run-001_eeg",
#    "sub-P004_ses-S002_task-T1_run-001_eeg",
#    "sub-P004_ses-S002_task-T1_run-002_eeg",
#    "sub-P005_ses-S001_task-T1_run-001_eeg",
#    "sub-P005_ses-S001_task-T1_run-002_eeg",
#    "sub-P005_ses-S001_task-T1_run-003_eeg",
#    "sub-P005_ses-S002_task-T1_run-001_eeg",
#    "sub-P006_ses-S001_task-T1_run-001_eeg",
#    "sub-P006_ses-S002_task-T1_run-001_eeg",
#    "sub-P006_ses-S002_task-T1_run-002_eeg",
#    "sub-P007_ses-S001_task-T1_run-001_eeg",
#    "sub-P007_ses-S001_task-T1_run-002_eeg",
#    "sub-P007_ses-S002_task-T1_run-001_eeg",
#    "sub-P008_ses-S001_task-T1_run-001_eeg",
#    "sub-P008_ses-S002_task-T1_run-001_eeg",
#    "sub-P008_ses-S002_task-T1_run-002_eeg",
#    "sub-P009_ses-S001_task-T1_run-001_eeg",
#    "sub-P009_ses-S002_task-T1_run-001_eeg",
#    "sub-P010_ses-S001_task-T1_run-001_eeg",
#    "sub-P010_ses-S002_task-T1_run-001_eeg"
# Enable to run files in Daniel's PC
# files = [
#     "sub-P004_ses-S001_task-T1_run-001_eeg",
#     "sub-P004_ses-S002_task-T1_run-002_eeg",
#     ]

#isolate subject IDs
subject_ids = [file.split('_')[0] for file in files]
unique_subject_ids = list(set(subject_ids))

# Preallocate variables
eeg_epochs = [None] * len(files)
settings = [None] * len(files)


# Import data
for f, file in enumerate(files):
    # Import EEG data
    eeg_epochs[f] = np.load(f"Data\{file}.npy")

    # Import settings
    with open(f"Data\{file}.json", "r") as file_object:
        settings[f] = json.load(file_object)


<string>:46: SyntaxWarning: invalid escape sequence '\{'
<string>:49: SyntaxWarning: invalid escape sequence '\{'
<>:46: SyntaxWarning: invalid escape sequence '\{'
<>:49: SyntaxWarning: invalid escape sequence '\{'
<string>:46: SyntaxWarning: invalid escape sequence '\{'
<string>:49: SyntaxWarning: invalid escape sequence '\{'
<>:46: SyntaxWarning: invalid escape sequence '\{'
<>:49: SyntaxWarning: invalid escape sequence '\{'
C:\Users\admin\AppData\Local\Temp\ipykernel_17704\1266126276.py:46: SyntaxWarning: invalid escape sequence '\{'
  eeg_epochs[f] = np.load(f"Data\{file}.npy")
C:\Users\admin\AppData\Local\Temp\ipykernel_17704\1266126276.py:49: SyntaxWarning: invalid escape sequence '\{'
  with open(f"Data\{file}.json", "r") as file_object:


## Compute the Power Spectral Density (PSD)

Compute the PSD of all the epoch for each file.

In [137]:
# PSD settings
window_size = 4 # Length of window for PSD [sec]

# Preallocate variables
eeg_f = [None] * len(files)
eeg_pxx = [None] * len(files)   # Preallocate to list in case not all files have the same number of channels

# Compute PSD for each file
for f,_ in enumerate(files):
    [eeg_f[f], eeg_pxx[f]] = signal.welch(
        x = eeg_epochs[f],
        fs = settings[f]["eeg_srate"],
        nperseg = window_size*settings[f]["eeg_srate"]
        )

## Visualize PSDs

In [33]:
# Plot settings
plot_psd = False    # Enable to see plots
f_limits = [5, 40]  # Frequency limits for the plots [min, max][Hz]
file_to_plot = 0    # Select index of file to be plotted 

if plot_psd:
    for s, stim in settings[file_to_plot]["stimuli"].items():
        fig, ax = plt.subplots(2,2)
        fig.suptitle(stim)

        for f, freq in settings[file_to_plot]["freqs"].items():
            fmask = (eeg_f[file_to_plot]>=f_limits[0]) & (eeg_f[file_to_plot]<=f_limits[1])
            temp_freq = eeg_f[file_to_plot][fmask]

            temp_mean = np.mean(eeg_pxx[file_to_plot][int(s), int(f), :, :], axis=0)[fmask]
            temp_sd = np.std(eeg_pxx[file_to_plot][int(s), int(f), :, :], axis=0)[fmask]

            row = int(f) // 2
            col = int(f) % 2
            ax[row, col].plot(temp_freq, temp_mean, '-')
            ax[row, col].set_title(f"{freq} Hz")
            
        
        ax[0,0].set_ylabel("PXX [$\mu$V$^2$/Hz]")
        ax[1,0].set_ylabel("PXX [$\mu$V$^2$/Hz]")
        ax[1,0].set_xlabel("Frequency [Hz]")
        ax[1,1].set_xlabel("Frequency [Hz]")
        plt.tight_layout()


<>:24: SyntaxWarning: invalid escape sequence '\m'
<>:25: SyntaxWarning: invalid escape sequence '\m'
<>:24: SyntaxWarning: invalid escape sequence '\m'
<>:25: SyntaxWarning: invalid escape sequence '\m'
C:\Users\admin\AppData\Local\Temp\ipykernel_17704\2451645341.py:24: SyntaxWarning: invalid escape sequence '\m'
  ax[0,0].set_ylabel("PXX [$\mu$V$^2$/Hz]")
C:\Users\admin\AppData\Local\Temp\ipykernel_17704\2451645341.py:25: SyntaxWarning: invalid escape sequence '\m'
  ax[1,0].set_ylabel("PXX [$\mu$V$^2$/Hz]")


# Compute SNR

Compute modified SNR as described in [Norcia et al., 2015.](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4581566/)

In [138]:
# Settings
noise_band = 0.5    # Single sided noise band [Hz]
nharms = 2          # Number of harmonics use [n]
db_out = True       # Boolean to get output in dB

# Preallocate variables
snr = [None] * len(files)

for f0,_ in enumerate(files):
    # Preallocate temp_snr
    temp_snr = np.zeros([
        len(settings[f0]["stimuli"]),
        len(settings[f0]["freqs"]),
        len(settings[f0]["ch_names"])
        ])

    # Compute SNR per stimuli and freq
    for s,stimuli in settings[f0]["stimuli"].items():
        for f1,freq in settings[f0]["freqs"].items():
                temp_snr[int(s),int(f1),:] = processing.ssvep_snr(
                    f = eeg_f[f0],
                    pxx = eeg_pxx[f0][int(s),int(f1),:,:],
                    stim_freq = float(freq),
                    noise_band = noise_band,
                    nharms = nharms,
                    db_out = db_out
                    )
                
    # Save temp SNR value
    snr[f0] = temp_snr


## Export SNR

Create a `pd.DataFrame` to store all the SNR values with appropriate labels and export to CSV file.

In [113]:
# Settings
save_snr = True     # Boolean to save SNRs to CSV
ch_subset = ["T7", "T8"]

# Preallocate empty list to store all dataFrames
dfs = []

for f0,file in enumerate(files):
    # Preallocate variables
    col_names = []

    # There is a more elegant way to do this transposing and reshaping the
    # numpy array, but I'll just leave it like this for now.
    # Might revisit if execution is too slow
    snr_shape = snr[f0].shape
    temp_snr = np.zeros((snr_shape[2], snr_shape[0]*snr_shape[1]))
    col_idx = 0
    for s,stimuli in settings[f0]["stimuli"].items():
        for f1,freq in settings[f0]["freqs"].items():
            temp_snr[:,col_idx] = snr[f0][int(s),int(f1),:]
            col_names.append(f"{stimuli} - {freq} Hz")

            col_idx += 1

    # Find indices of channel subset
    ch_subset_index = []
    row_names = []
    for channel in ch_subset:
        try:
            ch_subset_index.append(settings[f0]["ch_names"].index(channel))
            subject_id = file.split("_")[0]
            row_names.append(f"{subject_id} - {channel}")
        except ValueError:
            print(f"Trial {file} has no channel {channel} in dataset")
            
    # Create dataFrame for file
    dfs.append(
        pd.DataFrame(
            data = temp_snr[ch_subset_index, :],
            columns = col_names,
            index = row_names
            )
        )
    

# Concatenate all dataFrames
snr_df = pd.concat(dfs)

# Save SNRs to CSV
if (save_snr):
    snr_df.to_csv("Data\snr_complete_t.csv")

<>:51: SyntaxWarning: invalid escape sequence '\s'
<>:51: SyntaxWarning: invalid escape sequence '\s'
C:\Users\admin\AppData\Local\Temp\ipykernel_17704\1862226216.py:51: SyntaxWarning: invalid escape sequence '\s'
  snr_df.to_csv("Data\snr_complete_t.csv")


In [139]:
import pandas as pd
# Settings
save_snr = True  # Boolean to save SNRs to Excel
ch_subset = ["T7", "T8"]
# Create a directory to store the Excel file
import os
output_directory = "Data\excel_output_new\\t_channels\\P008"
os.makedirs(output_directory, exist_ok=True)
averaged_dfs = []  # List to store averaged DataFrames
all_data = {}  # Dictionary to store all data

for f0, file in enumerate(files):
    # Preallocate variables
    col_names = []
    sub_ses_info = "_".join(file.split("_")[:2])
    info_parts = sub_ses_info.split("_")
    ses_info = info_parts[1]
    snr_shape = snr[f0].shape
    temp_snr = np.zeros((snr_shape[2], snr_shape[0] * snr_shape[1]))
    col_idx = 0
    for s, stimuli in settings[f0]["stimuli"].items():
        for f1, freq in settings[f0]["freqs"].items():
            temp_snr[:, col_idx] = snr[f0][int(s), int(f1), :]
            col_names.append(f"{stimuli} - {freq} Hz")
            col_idx += 1
    # Find indices of channel subset
    ch_subset_index = []
    row_names = []
    for channel in ch_subset:
        try:
            ch_subset_index.append(settings[f0]["ch_names"].index(channel))
            row_names.append(f"{id} - {channel}")
        except ValueError:
            print(f"Channel {channel} is not in dataset")
    # Create a DataFrame for the current file
    snr_df = pd.DataFrame(
        data=temp_snr[ch_subset_index, :],
        columns=col_names,
        index=row_names
    )
    # Transpose the DataFrame
    snr_df = snr_df.T
    # Calculate the row-wise average
    averaged_snr = snr_df.mean(axis=1)
    # Convert the averaged data to a DataFrame
    averaged_df = pd.DataFrame(averaged_snr, columns=[ses_info])
    averaged_dfs.append(averaged_df)
    # Store the DataFrame in the dictionary
    all_data[sub_ses_info] = averaged_df

# Combine all DataFrames into a single DataFrame
combined_df = pd.concat(all_data, axis=1)
combined_df = combined_df.T

#remove 2Hz columns
columns_to_drop = [0,4,8,12,16,20] #,24,28,32,36,40,44,48,52,56]
combined_df_fix = combined_df.drop(combined_df.columns[columns_to_drop], axis = 1)


# Define the Excel file name
excel_filename = os.path.join(output_directory, "averaged_snr_t_P008_S002_R001.xlsx")

# Save the combined DataFrame to an Excel file
if save_snr:
    combined_df_fix.to_excel(excel_filename)

<>:7: SyntaxWarning: invalid escape sequence '\e'
<>:7: SyntaxWarning: invalid escape sequence '\e'
C:\Users\admin\AppData\Local\Temp\ipykernel_17704\1120255435.py:7: SyntaxWarning: invalid escape sequence '\e'
  output_directory = "Data\excel_output_new\\t_channels\\P008"
